<a href="https://colab.research.google.com/github/pablojrios/fluence_maps/blob/master/tf2_transfer_learning_gamma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning with a pretrained ConvNet

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

%cd '/content'
if os.path.exists('fluence_maps'):
  !rm -fr fluence_maps

## Install required dependencies
!pip install -q pydicom
## to support ResNet18 and ResNet34
!pip install image-classifiers
## https://github.com/tensorflow/addons/issues/2251
!pip install -U tensorflow-addons

GIT_USERNAME = "pablojrios"
GIT_TOKEN = "1d88a0b85d2b00a03796e4d8b7e5f7b249b12f9b"
!git clone -s https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/fluence_maps.git

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
from random import shuffle, randrange
import random
import tensorflow_addons as tfa

In [ ]:
import tensorflow as tf
print('Tensorflow version = {}, addons version = {}'.format(tf.__version__, tfa.__version__))
print('Executing eagerly = {}'.format(tf.executing_eagerly()))
keras = tf.keras

# To support ResNet18 and ResNet34 for tensorflow.keras
from classification_models.tfkeras import Classifiers

import cv2 # to perform data augmentation
print('OpenCV version = {}'.format(cv2.__version__))

datetime.now().strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ===============================================DEFINE YOUR ARGUMENTS=================================================
ARG_DATASET_DIR='/content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata'
# folder under ARG_DATASET_DIR path.
ARG_TFDATASET_FOLDER='tfds.2019.localnorm.ovs95'
ARG_RANDOM_SEED = 23456
# if False only training and validation partition are created.
ARG_TEST_PARTITION=False
# number of continuous epochs without improvement on validation
ARG_EPOCHS_WO_IMPROVEMENT=30
# maximum fine-tuning epochs
ARG_MAX_FINE_TUNING_EPOCHS=200
ARG_RESNET_NETWORK=True
ARG_DATA_AUGMENTATION=True
# perform data augmentation of images with a gamma value lower than gamma_augment
ARG_GAMMA_AUGMENT=95.0
# set this value based on the ARG_OVERSAMPLING_FACTOR value in tf2_oversampling_dicom_files.py
# 1.0 means transform each and every image, with a lower value than 1.0 means that training will include unmodified (not transformed)
# images.
ARG_AUGMENT_PROBABILITY=0.75
 allows to train on ex)
add_regularizers=False
## Fine-tune from this layer onwards
# fine_tune_at = 281 # InceptionV3, fine-tuning
# fine_tune_at = 102 # InceptionV3, not so fine-tuning
# fine_tune_at = 12 # VGG16
fine_tune_at = 47 # resnet18 stage3
# fine_tune_at = 74 # resnet34 stage3
# fine_tune_at = 129 # resnet34 stage4

## Data preprocessing

### Data download

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

tf.random.set_seed(ARG_RANDOM_SEED)

In [ ]:
def _tfrecord_dataset_type_from_folder(folder, dataset_type, ext='.tfrecords'):
    tfrecords = [os.path.join(folder, n)
                 for n in os.listdir(folder) if n.startswith(dataset_type) and n.endswith(ext)]
    return tf.data.TFRecordDataset(tfrecords)

tfdataset_dir = os.path.join(ARG_DATASET_DIR, ARG_TFDATASET_FOLDER)
raw_train = _tfrecord_dataset_type_from_folder(tfdataset_dir, 'train')
raw_validation = _tfrecord_dataset_type_from_folder(tfdataset_dir, 'validation')
if ARG_TEST_PARTITION:
    raw_test = _tfrecord_dataset_type_from_folder(tfdataset_dir, 'test')

The resulting `tf.data.Dataset` objects contain `(image, label)` pairs where the images have variable shape and 3 channels, and the label is a scalar.

In [ ]:
print(raw_train)
print(raw_validation)
if ARG_TEST_PARTITION:
    print(raw_test)

In [ ]:
print(f'Number of images in validation part = {sum(1 for _ in raw_validation)}.')

## Data Augmentation

### Center Crop

In [ ]:
import math
def crop(im, r, c, target_r, target_c): return im[r:r+target_r, c:c+target_c]

# random crop to the original size
def random_crop(x, r_pix=8):
    """ Returns a random crop"""
    r, c,*_ = x.shape
    c_pix = round(r_pix*c/r)
    rand_r = random.uniform(0, 1)
    rand_c = random.uniform(0, 1)
    start_r = np.floor(2*rand_r*r_pix).astype(int)
    start_c = np.floor(2*rand_c*c_pix).astype(int)
    return crop(x, start_r, start_c, r-2*r_pix, c-2*c_pix)

def center_crop(x, r_pix=8):
    r, c,*_ = x.shape
    c_pix = round(r_pix*c/r)
    return crop(x, r_pix, c_pix, r-2*r_pix, c-2*c_pix)


def rotate_cv(im, deg, mode=cv2.BORDER_REFLECT, interpolation=cv2.INTER_AREA):
    """ Rotates an image by deg degrees"""
    r,c,*_ = im.shape
    print(f"rotate_cv: {r},{c}")
    M = cv2.getRotationMatrix2D((c/2,r/2),deg,1)
    return cv2.warpAffine(im,M,(c,r), borderMode=mode, 
                          flags=cv2.WARP_FILL_OUTLIERS+interpolation)

### Translation

In [ ]:
def random_translation(x, t_pix=24):
    """ Returns a random translation"""
    rows, cols, *_ = x.shape
    rand_r = random.uniform(0, 1)
    rand_c = random.uniform(0, 1)
    t_r = np.floor(rand_r*t_pix).astype(int)
    t_c = np.floor(rand_c*t_pix).astype(int)
    # transformation T does shift (t_r, t_c)
    # [a0, a1, a2, b0, b1, b2, c0, c1], then it maps the output point
    # (x, y) to a transformed input point
    # (x', y') = ((a0 x + a1 y + a2) / k, (b0 x + b1 y + b2) / k),
    # where k = c0 x + c1 y + 1
    T = [1, 0, t_r, 0, 1, t_c, 0, 0]
    return tfa.image.transform (x, T)

In [ ]:
def rescale_0_to_1(image):
    """
    Rescale image to [0, 1].
    """
    # Image must be casted to float32 first
    image = tf.cast(image, tf.float32)
    image = (image/255.0)
    return image


def rescale_min_1_to_1(image):
    """
    Rescale image to [-1, 1].

    :param image:
        Required. Image tensor.

    :return:
        Scaled image.
    """
    # Image must be casted to float32 first
    image = tf.cast(image, tf.float32)
    # Rescale image from [0, 255] to [0, 2], and by substracting -1 we rescale to [-1, 1].
    image = (image/127.5) - 1
    return image

In [ ]:
def _parse_image_function(example_proto, img_size, normalization_fn, data_augmentation=False, augment_probability=1.0):
    # Create a dictionary describing the features.
    image_feature_description = {"image/filename": tf.io.FixedLenFeature((), tf.string),
                "image/encoded": tf.io.FixedLenFeature((), tf.string),
                "image/format": tf.io.FixedLenFeature((), tf.string),
                "image/gamma_index": tf.io.FixedLenFeature((), tf.float32),
                "image/height": tf.io.FixedLenFeature((), tf.int64),
                "image/width": tf.io.FixedLenFeature((), tf.int64)}
    
    def image_augment(image):
        radian = ((np.random.random()-.50)*20 / 360) * np.pi
        # tfa.image.transform_ops.rotate(image, radian) is an alias
        image = tfa.image.rotate(image, radian)
        # image = tf.image.random_flip_left_right(image)
        image = random_translation(image)
        return image

    # Now, globally set everything to run eagerly
    # The following doesn't set to eager mode:
    # UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set,
    # this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
    # tf.config.run_functions_eagerly(True)

    # Executing eagerly = False here!
    # numpy is only supported in eager mode. If you are in graph mode, it will not be supported.
    # In eager execution the shape is always fully-known.
    # print('Executing eagerly = {}'.format(tf.executing_eagerly()))

    # Parse the input tf.Example proto using the dictionary above.
    parsed = tf.io.parse_single_example(example_proto, image_feature_description)
    
    image = tf.image.decode_jpeg(parsed["image/encoded"], channels=3)
    # print(type(image), image.shape, image.dtype) # <class 'tensorflow.python.framework.ops.Tensor'> (None, None, 3) <dtype: 'uint8'>

    gamma = tf.cast(
        parsed["image/gamma_index"],
        tf.float32)
    # print(type(gamma), gamma.shape, gamma.dtype) # <class 'tensorflow.python.framework.ops.Tensor'> () <dtype: 'float32'>

    image = normalization_fn(image)

    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    print(type(image), image.shape, image.dtype)

    if data_augmentation and augment_probability >= random.uniform(0, 1):
        gamma_augment = tf.constant(ARG_GAMMA_AUGMENT)
        image = tf.cond(tf.math.less(gamma, gamma_augment)
                        , lambda: image_augment(image)
                        , lambda: image)

    # normalizo antes de transformar
    # image = normalization_fn(image)
    
    #label is a tensor of an array of single tf.int64 arrays.
    #label = tf.cast(
    #    tf.reshape(parsed["image/class/label"], [-1]),
    #    tf.int64)

    # assert tf.executing_eagerly() FAILS
    # parsed["image/filename"] is a Tensor and not an EagerTensor because we are in a map function,
    # because in 2.0, code inside Datasets maps is turned into a subgraph for speed, just as it was in 1.x eager
    # execution. You generally want to avoid Python inside your data pipeline.
    # So, if I invoke parsed["image/filename"].numpy().decode('utf-8') to get the filename string the error
    # "AttributeError: 'Tensor' object has no attribute 'numpy'" is thrown, hence I return a tensor.
    filename = parsed["image/filename"]
            
    return image, gamma, filename

In [ ]:
IMG_SIZE = 256 # All images will be resized to 256x256
normalization_fn = rescale_0_to_1 # rescale_min_1_to_1
# normalization_fn = tf.image.per_image_standardization # loss y mae en validación reportan números muy grandes,
# no así en training.

In [ ]:
num_workers = 8

# assert tf.executing_eagerly()
if ARG_DATA_AUGMENTATION:
    print("Training with image augmentation.")
    
train = raw_train.map(lambda e: _parse_image_function(e, IMG_SIZE, normalization_fn, ARG_DATA_AUGMENTATION, ARG_AUGMENT_PROBABILITY),
                      num_parallel_calls=num_workers)
validation = raw_validation.map(lambda e: _parse_image_function(e, IMG_SIZE, normalization_fn),
                                num_parallel_calls=num_workers)
if ARG_TEST_PARTITION:
    test = raw_test.map(lambda e: _parse_image_function(e, IMG_SIZE, normalization_fn),
                        num_parallel_calls=num_workers)

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = 2 * BATCH_SIZE

In [ ]:
print('Executing eagerly = {}'.format(tf.executing_eagerly()))

# <PrefetchDataset shapes: ((None, 224, 224, 3), (None, 1)), types: (tf.float32, tf.int64)>
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER_SIZE)
print(train_batches)
validation_batches = validation.batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER_SIZE)
print(validation_batches)
# <BatchDataset shapes: ((None, 224, 224, 3), (None, 1)), types: (tf.float32, tf.int64)>
if ARG_TEST_PARTITION:
    test_batches = test.batch(BATCH_SIZE)
    print(test_batches)

Inspect a batch of data:

In [ ]:
for image_batch, label_batch, filename_batch in train_batches.take(1):
    pass

image_batch.shape, label_batch.shape, filename_batch.shape

In [ ]:
# display 2nd image in the batch
gamma_value = label_batch[7].numpy()
filename = filename_batch[7].numpy().decode('utf-8')
print(f'gamma={gamma_value:2.2f}, filename={filename}')
# if pixel values are float they have to be in [0, 1] range, if they are integer they have to be in the [0, 255] range,
# else pixel values are truncated.
im = image_batch[7].numpy()
plt.imshow(im)
print("image shape = {}".format(im.shape))

## Create the base model from the pre-trained convnets

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

if ARG_RESNET_NETWORK:
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    base_model = ResNet18(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)

else:
    ## Create the base model from the pre-trained model MobileNet V2
    #base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
    #                                               # alpha=1.4,
    #                                               include_top=False,
    #      

    
    # Create the base model from the pre-trained model Inception V3

    # When include_top=True and weights=None (random initialization), see below:
    # __________________________________________________________________________________________________
    # avg_pool (GlobalAveragePooling2 (None, 2048)         0           mixed10[0][0]                    
    # __________________________________________________________________________________________________
    # predictions (Dense)             (None, 1000)         2049000     avg_pool[0][0]                   
    # ==================================================================================================        

    #base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
    #                                               weights='imagenet',
    #                                               include_top=False)

    #base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
    # 


    base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                                   weights='imagenet',
                                                   include_top=False)

This feature extractor converts each `160x160x3` image into a `5x5x1280` block of features. See what it does to the example batch of images:

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Feature extraction
In this step, you will freeze the convolutional base created from the previous step and to use as a feature extractor. Additionally, you add a classifier on top of it and train the top-level classifier.

### Freeze the convolutional base

It is important to freeze the convolutional base before you compile and train the model. Freezing (by setting layer.trainable = False) prevents the weights in a given layer from being updated during training. MobileNet V2 has many layers, so setting the entire model's trainable flag to False will freeze all the layers.

In [ ]:
if not ARG_RESNET_NETWORK:
    base_model.trainable = False
else:
    # resnet
    for layer in base_model.layers:
            if layer.__class__.__name__ != "BatchNormalization":
                layer.trainable = False
            else:
                print(f"{layer.name} ({layer.__class__.__name__})")

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

In [ ]:
%cd '/content/fluence_maps'
from add_regularization import add_regularization
# adds a tf.keras.regularizers.l2(0.0001)
#"kernel_regularizer":{
#                        "class_name": "L1L2",
#                        "config": {
#                            "l1": 0,
#                            "l2": 0.0001
#                        }
#                     }
if add_regularizers:
    base_model = add_regularization(base_model)

for layer in base_model.layers:
    for attr in ['kernel_regularizer']:
        if hasattr(layer, attr):
            if getattr(layer, attr) is None:
                print('layer {} has no regularizer.'.format(layer.name))
            else:
                print('layer {} has a regularizer {}.'.format(layer.name, getattr(layer, attr)))


In [ ]:
# from render_json import RenderJSON

# RenderJSON(base_model.to_json())

In [ ]:
# display the weights of some layers
l = 1
for layer in base_model.layers:
    # print('layer {:3d}, name: {}'.format(l, layer.name))
    if layer.name == "expanded_conv_project":
        weights = layer.get_weights()
        print(layer.get_config(), weights, weights[0].shape)
    l += 1

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Stack the feature extractor, and these two layers using a `tf.keras.Sequential` model for network architectures other than ResNet:

In [ ]:
if not ARG_RESNET_NETWORK:
    # Pablo March 10: add sigmoid
    # WARNING: adding the activation function causes loss to keep close to 0.5 and does not decrease.
    # prediction_layer = keras.layers.Dense(1, activation='sigmoid') # para obtener probabilidades y no logits

    # https://keras.io/applications/#fine-tune-inceptionv3-on-a-new-set-of-classes
    # https://stackoverflow.com/questions/58627411/how-to-use-inception-network-for-regression

    # let's add a fully-connected layer
    fc1 = keras.layers.Dense(512, activation='relu')
    #fc2 = keras.layers.Dense(512, activation='relu')
    #dropout = keras.layers.Dropout(rate=0.05) # no funciona
    # and a linear output layer (regression)
    prediction_layer = keras.layers.Dense(1, activation='linear')
    # and a logistic layer -- let's say we have 200 classes (classification)
    # prediction_layer = Dense(200, activation='softmax')(x)

    # Up to March 27, 2020
    # prediction_layer = keras.layers.Dense(1)
    prediction_batch = prediction_layer(feature_batch_average)
    print(prediction_batch.shape)

    # Now stack the feature extractor, and these two layers using a `tf.keras.Sequential` model:
    model = tf.keras.Sequential([
      base_model,
      global_average_layer,
      fc1,
      #dropout,
      #fc2,
      prediction_layer
    ])
    
else:
    avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    out = tf.keras.layers.Dense(512, activation="relu")(avg) # CHANGE to use avg pool only or avg pool + max pool
    out = tf.keras.layers.BatchNormalization()(out) # bn after dropout or dense layer helps!
    prediction_layer = tf.keras.layers.Dense(1, activation='linear')(out)
    model = keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

### Compile the model

You must compile the model before training it.  Since there are two classes, use a binary cross-entropy loss with `from_logits=True` since the model provides a linear output.

In [ ]:
base_learning_rate = 0.001
# mse = square(y_true - y_pred)
# mae = loss = abs(y_true - y_pred)
# mape = 100 * abs(y_true - y_pred) / y_true
# mae y mape son similares, no iguales, por eso tomo MAE que es el promedio de la diferencia absoluta entre el
# gamma real y el gamma predicho
#optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate, momentum=0.95)
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mse'])

In [ ]:
model.summary()

### Train the model

In [ ]:
num_train = sum(1 for _ in raw_train)
num_val = sum(1 for _ in raw_validation)
print(f'Number of images in train partition: {num_train}, in validation: {num_val}.')
if ARG_TEST_PARTITION:
    num_test = sum(1 for _ in raw_test)
    print(f'Number of images in test partiton: {num_test}.')

In [ ]:
initial_epochs = 10
steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps=20

# projects out just the first two components.
tmp_validation_batches = validation_batches.map(lambda image, gamma, filename: (image, gamma))
print(tmp_validation_batches)

loss0 = mse0 = 0
loss0, mse0 = model.evaluate(tmp_validation_batches, steps = validation_steps)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial mape: {:.2f}".format(mse0))

In [ ]:
# projects out just the first two components.
tmp_train_batches = train_batches.map(lambda image, gamma, filename: (image, gamma))
print(tmp_train_batches)

In [ ]:
# Implement callback function to stop training
class MyCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, wait_epochs):
        self.__wait_epochs = wait_epochs
        self.__latest_peak_mae = 4.0
        self.__waited_epochs = 0
        self.__saved_model_file = None
        self.__saved_model = None
        
    def stopTraining(self, epoch, val_mae):
        stop_early = False
        # check for early stop
        if val_mae < self.__latest_peak_mae:
            self.__latest_peak_mae = val_mae
            print(f"\nNew peak val_mae reached: {val_mae:6.4}")

            t = time.time()
            dir = os.path.join(ARG_DATASET_DIR, "models")
            save_model_path = "{}/{}.h5".format(dir, int(t))
            print(save_model_path)
            # Either 'tf' or 'h5', indicating whether to save the model to Tensorflow SavedModel or HDF5. Defaults to 'tf' 
            # in TF 2.X, and 'h5' in TF 1.X.            
            model.save(save_model_path, save_format='h5')
            # borro el archivo del modelo anterior
            if self.__saved_model_file is not None:
                os.remove(self.__saved_model_file)
            self.__saved_model_file = save_model_path
            reloaded_model = tf.keras.models.load_model(save_model_path)
            self.__saved_model = reloaded_model

        if val_mae > self.__latest_peak_mae:
            # Si llevo N+ epochs sin mejora
            if self.__waited_epochs >= self.__wait_epochs:
                print("\nStopping early at epoch {0} with saved peak mae {1:10.8}"
                      .format(epoch + 1, self.__latest_peak_mae))
                stop_early = True
            
            self.__waited_epochs += 1
            
        else:
            self.__latest_peak_mae = val_mae
            # Reset waited epochs.
            self.__waited_epochs = 0
            
        return stop_early
        
    def on_epoch_end(self, epoch, logs={}):
        # print('\nTraining: epoch {} ends at {}'.format(epoch, datetime.now().strftime("%d/%m/%Y %H:%M:%S")))
        if self.stopTraining(epoch, logs.get('val_loss')):
            self.model.stop_training = True

    @property
    def saved_model_file(self):
        return self.__saved_model_file
    
    @property
    def saved_model(self):
        return self.__saved_model

    def reset_waited_epochs(self):
        self.__waited_epochs = 0;
    

# Instantiate a callback object
callbackObj = MyCallback(ARG_EPOCHS_WO_IMPROVEMENT)

In [ ]:
history = model.fit(tmp_train_batches,
                    epochs=initial_epochs,
                    validation_data=tmp_validation_batches)

### Learning curves

Let's take a look at the learning curves of the training and validation accuracy/loss when using the MobileNet V2 base model as a fixed feature extractor.

In [ ]:
print(history.epoch)

In [ ]:
# Pablo Feb 25:
# Python never implicitly copies objects. When you set list2 = list1, you are making them refer to the same exact
# list object, so when you mutate it, all references to it keep referring to the object in its current state.
mae = history.history['mse'].copy()
val_mae = history.history['val_mse'].copy()

loss = history.history['loss'].copy()
val_loss = history.history['val_loss'].copy()

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(mae, label='Training MSE')
plt.plot(val_mae, label='Validation MSE')
plt.legend(loc='upper right')
plt.ylabel('MSE')
plt.ylim([0,50])
plt.title('Training and Validation MSE')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.ylim([0,10])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Fine tuning
In the feature extraction experiment, you were only training a few layers on top of a VGG16 base model. The weights of the pre-trained network were **not** updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic feature maps to features associated specifically with the dataset.

Note: This should only be attempted after you have trained the top-level classifier with the pre-trained model set to non-trainable. If you add a randomly initialized classifier on top of a pre-trained model and attempt to train all layers jointly, the magnitude of the gradient updates will be too large (due to the random weights from the classifier) and your pre-trained model will forget what it has learned.

Also, you should try to fine-tune a small number of top layers rather than the whole VGG16 model. In most convolutional networks, the higher up a layer is, the more specialized it is. The first few layers learn very simple and generic features that generalize to almost all types of images. As you go higher up, the features are increasingly more specific to the dataset on which the model was trained. The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.

### Un-freeze the top layers of the model


All you need to do is unfreeze the `base_model` and set the bottom layers to be un-trainable. Then, you should recompile the model (necessary for these changes to take effect), and resume training.

In [ ]:
base_model.trainable = True

# Pablo Feb 25: con este loop me aseguro que las capas que en la próxima celda *NO* pongo como layer.trainable = False sean
# trainable.
l = 1
for layer in base_model.layers:
    print('layer {:3d}, name: {}'.format(l, layer.name))
    if layer.__class__.__name__ != "BatchNormalization":
        layer.trainable =  True
    l += 1

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    if layer.__class__.__name__ != "BatchNormalization":
        layer.trainable = False

### Compile the model

Compile the model using a much lower learning rate.

In [ ]:
# This function keeps the learning rate at 0.001 for the first ten epochs
# and decreases it exponentially after that.
def lr_scheduler(epoch):
    lr = base_learning_rate
    #k = 0.15
    #if epoch >= 10 and epoch <= 40:
    #    lr = tf.math.exp(k * (10 - epoch)) * base_learning_rate
    #    print("\nlearning rate: %.8f"%(lr))
    if epoch >= 10 and epoch <= 30:
        lr = base_learning_rate/10
    elif epoch > 30 and epoch <= 60:
        lr = base_learning_rate/100
    elif epoch > 60:
        lr = base_learning_rate/500
    print("\nlearning rate: %.6f"%(lr))
    
    return lr
    
    
callbackLR = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [ ]:
# optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10, momentum=0.95)
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10, epsilon=1e-08, amsgrad=False)
model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
              optimizer = optimizer,
              metrics=['mse'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Continue training the model

In [ ]:
# Crear el path donde se va a ir guardando el mejor mejor modelo en training.
#dirName = "./tmp/saved_models/"
#if not os.path.exists(dirName):
#    os.makedirs(dirName)
#    print("Directory" , dirName ,  "Created.")
#else:    
#    print("Directory" , dirName ,  "already exists.")

If you trained to convergence earlier, this step will improve your accuracy by a few percentage points.

In [ ]:
fine_tune_epochs = ARG_MAX_FINE_TUNING_EPOCHS
total_epochs =  (history.epoch[-1]+1) + fine_tune_epochs

history_fine = model.fit(tmp_train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1]+1,
                         validation_data=tmp_validation_batches,
                         callbacks=[callbackObj, callbackLR])

In [ ]:
print(history_fine.epoch)

In [ ]:
mae += history_fine.history['mse']
val_mae += history_fine.history['val_mse']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
mae = mae[1:]
val_mae = val_mae[1:]
loss = loss[1:]
val_loss = val_loss[1:]

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(mae, label='Training MSE')
plt.plot(val_mae, label='Validation MSE')
plt.ylim([0, 4])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Fine Tuning starts')
plt.legend(loc='upper right')
plt.ylabel('MSE')
plt.title('Training and Validation MSE')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 5])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Fine Tuning Starts')
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Summary:

* **Using a pre-trained model for feature extraction**:  When working with a small dataset, it is a common practice to take advantage of features learned by a model trained on a larger dataset in the same domain. This is done by instantiating the pre-trained model and adding a fully-connected classifier on top. The pre-trained model is "frozen" and only the weights of the classifier get updated during training.
In this case, the convolutional base extracted all the features associated with each image and you just trained a classifier that determines the image class given that set of extracted features.

* **Fine-tuning a pre-trained model**: To further improve performance, one might want to repurpose the top-level layers of the pre-trained models to the new dataset via fine-tuning.
In this case, you tuned your weights such that your model learned high-level features specific to the dataset. This technique is usually recommended when the training dataset is large and very similar to the original dataset that the pre-trained model was trained on.


In [ ]:
!ls -l "/content/drive/MyDrive/Healthcare/Radioterapia/data/ciolaplata/models/"

In [ ]:
print(f'Best saved model file = {callbackObj.saved_model_file}')
saved_model = callbackObj.saved_model

In [ ]:
# Evaluate the saved model on the train set which differs from values reported during training
loss, mse = saved_model.evaluate(tmp_train_batches, verbose=2)
print("Saved model, train loss: {:5.4f}".format(loss))
print('Saved model, train mse: {:5.4f}'.format(mse))

# Evaluate the saved model on the validation set which differs from values reported during training
loss, mse = saved_model.evaluate(tmp_validation_batches, verbose=2)
print("\nSaved model, validation loss: {:5.4f}".format(loss))
print('Saved model, validation mse: {:5.4f}'.format(mse))

In [ ]:
#result_batch = model.predict(tmp_train_batches)
#reloaded_result_batch = reloaded_model.predict(tmp_train_batches)
#print(abs(reloaded_result_batch - result_batch).max())
#np.testing.assert_allclose(result_batch, reloaded_result_batch, rtol=1e-6, atol=1e-6)

In [ ]:
# projects out just the first two components.
if ARG_TEST_PARTITION:
    tmp_test_batches = test_batches.map(lambda image, gamma, filename: (image, gamma))
    print(tmp_test_batches)

    # Evaluate the reloaded model on the test set
    loss, mae = saved_model.evaluate(tmp_test_batches, verbose=1)
    print("\n\nSaved model, test loss: {:5.4f}".format(loss))
    print('Saved model, test mae: {:5.4f}'.format(mae))

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')